In [1]:
# Es la única que necesitamos realmente
!pip install igraph
!pip install nolds

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 2.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Ejecutar

Estas celdas son las responsables de guardar la información en JSON de las métricas

In [9]:
# TODO: Buscar la dimensión fractal con el método de box counting
import os
import time
import json

import numpy as np

import igraph as ig
import networkx as nx

import nolds
import scipy.sparse as sp
from scipy.stats import entropy


root: str = "/content/drive/MyDrive/BETA SKELETON LUNA Y CHUCHI"
graphs_root: str = f"{root}/grafos"

assert os.path.exists(graphs_root)

Pienso que la mejor manera de guardar todas las métricas puede ser por medio de un JSON (básicamente un diccionario, o un par clave valor), y pienso que quizás sea la mejor manera.

In [7]:
def calculate_metrics(g: ig.Graph, save_dict: dict,
                      laplacian_path: str, cutoff: int | None = None) -> None:
    """
    Aquí se calculan literalmente todas las métricas que se necesitan para el grafo `g`,
    y son guardadas en un diccionario (en `save_dict`) todos los valores correspondientes
    de cada métrica.

    Args:
        g (igraph.Graph): Grafo no dirigido el cual se le sacarán las métricas, es importante anotar
            que las aristas deben tener un peso el cual es llamado `distance`.
        save_dict (dict): Diccionario en el cual se guardarán el resultado de las métricas.
        laplacian_path (str): Nombre del archivo (junto a su ubicación) donde se guardará el
            resultado de la matriz laplaciana. La extensión de este tipo de archivo debe ser .npz.
        cutoff (int | None): Este parametro es esencial para especificar qué tan aproximado se
            quiere que las métricas de centralidad de betweenness y closeness sean. Por defecto,
            `cutoff=None`, lo que significa que no se hace ninguna aproximación, es decir, que se
            retorna el valor real.
    Returns:
        None: La función no retorna nada, únicamente devuelve el diccionario completo con las
            métricas que se calcularon y con la matriz laplaciana guardada en la dirección que se
            especificó previamente.
    """

    def global_efficiency(g: ig.Graph) -> float:
        shortest_paths = g.distances()
        efficiency = 0.0
        n = len(g.vs)

        for i in range(n):
            for j in range(i+1, n):
                if shortest_paths[i][j] > 0:
                    efficiency += 1/shortest_paths[i][j]

        efficiency /= (n * (n - 1) / 2)

        return efficiency

    def local_efficiency(g: ig.Graph, v: ig.Vertex) -> float:
        neighbors = g.neighbors(v)
        subgraph = g.subgraph(neighbors)

        if len(neighbors) <= 1:
            return 0

        return global_efficiency(subgraph)

    # Métricas de eficiencia
    if 'global_efficiency' not in save_dict:
        print("Calcular 'global_efficiency'")
        try:
            efficiency = global_efficiency(g)
            save_dict['global_efficiency'] = efficiency
        except Exception as e:
            print(repr(e))

    if 'local_efficiencies' not in save_dict:
        print("Calcular 'local_efficiencies'")
        try:
            local_efficiencies = [local_efficiency(g, v.index) for v in g.vs]
            avg_local_efficiency = sum(local_efficiencies) / len(local_efficiencies)

            save_dict['local_efficiencies'] = local_efficiencies
            save_dict['avg_local_efficiency'] = avg_local_efficiency
        except Exception as e:
            print(repr(e))

    # Calcula la centralidad de grado y normalízala
    degree = g.degree()
    degree_distribution = np.array(degree) / sum(degree)

    # Entropia
    if 'entropy' not in save_dict:
        print("Calcular 'entropy")
        graph_entropy = entropy(degree_distribution)
        save_dict['entropy'] = graph_entropy

    # Dimensión fractal (box counting)
    if 'fractal_dimension' not in save_dict:
        print("Calcular 'fractal_dimension'")
        # Falta buscar la función todavía, pero se aplicaría
        # sobre `degree_distribution`
        pass

    # Hurst
    if 'hurst' not in save_dict:
        print("Calcular 'hurst'")
        hurst = nolds.hurst_rs(degree_distribution, fit='poly')
        save_dict['hurst'] = hurst

    # Centralidad
    if 'closeness' not in save_dict:
        print("Calcular 'closeness'")
        closeness = g.closeness(weights='distance', normalized=True, cutoff=cutoff)
        save_dict['closeness'] = closeness

    if 'betweenness' not in save_dict:
        print("Calcular 'betweenness'")
        betweenness = g.betweenness(directed=False, weights='distance', cutoff=cutoff)
        save_dict['betweenness'] = betweenness

    if 'eigenvector' not in save_dict:
        print("Calcular 'eigenvector'")
        eigenvector = g.eigenvector_centrality(directed=False, weights='distance', scale=False)
        save_dict['eigenvector'] = eigenvector

    if 'convergence' not in save_dict:
        print("Calcular 'convergence'")
        convergence = g.convergence_degree()
        save_dict['convergence'] = convergence

    # Matriz laplaciana (La matriz laplaciana no se demora ada)
    G = g.to_networkx()
    laplacian = nx.laplacian_matrix(G, weight='distance')

    # Guardar matriz si es un archivo .npz
    if os.path.splitext(laplacian_path)[1] == '.npz':
        sp.save_npz(laplacian_path, laplacian)
        save_dict['laplacian'] = laplacian_path


In [8]:
save = True
cont = 1
snapnum = "001"  # @param

# Estas iteraciones son únicamente válidas cuando el snapnum va de 0 hasta 4
if os.path.exists(f'{graphs_root}/metrics.json'):
    with open(f'{graphs_root}/metrics.json', 'r') as json_data:
        metrics = json.load(json_data)
else:
    metrics = {}

try:
    # Iterar sobre cada simulación con grafos creados
    for simu in os.listdir(graphs_root):
        tmp_path = f'{graphs_root}/{simu}'
        if not os.path.isdir(tmp_path):
            continue

        if simu not in metrics:
            metrics[simu] = {}

        # Se podria crear una función para evitar duplicar código
        if simu == "randoms":
            # Si no hay un grafo para el snapnum asignado, saltarlo
            graph_path = f'{snapdir}/{simu}_{snapnum}.graphml'
            if not os.path.exists(graph_path):
                continue

            if snapnum not in metrics[simu][i]:
                metrics[simu][i][snapnum] = {}

            # Calcular las métricas del grafo
            laplacian_path = f'{snapdir}/laplacian_{simu}_{snapnum}.npz'
            g = ig.Graph.Read_GraphML(graph_path)
            print(f'{cont}. {graph_path} -- Vertices {len(g.vs):,} -- Aristas {len(g.es):,}')

            t0 = time.perf_counter()
            calculate_metrics(g, metrics[simu][i][snapnum], laplacian_path)
            tf = time.perf_counter() - t0

            print(f'Tiempo: {tf} segundos\n')

            cont += 1

        # Iterar sobre las realizaciones
        for i in os.listdir(tmp_path):
            snapdir = f'{tmp_path}/{i}'
            if i not in metrics[simu]:
                metrics[simu][i] = {}

            # Si no hay un grafo para el snapnum asignado, saltarlo
            graph_path = f'{snapdir}/{simu}_{snapnum}.graphml'
            if not os.path.exists(graph_path):
                continue

            if snapnum not in metrics[simu][i]:
                metrics[simu][i][snapnum] = {}

            # Calcular las métricas del grafo
            laplacian_path = f'{snapdir}/laplacian_{simu}_{snapnum}.npz'
            g = ig.Graph.Read_GraphML(graph_path)
            print(f'{cont}. {graph_path} -- Vertices {len(g.vs):,} -- Aristas {len(g.es):,}')

            t0 = time.perf_counter()
            calculate_metrics(g, metrics[simu][i][snapnum], laplacian_path)
            tf = time.perf_counter() - t0

            print(f'Tiempo: {tf} segundos\n')

            cont += 1

        print()
except KeyboardInterrupt:
    save = False
except Exception as e:
    pass

if save:
    print('Guardando métricas...')
    with open(f'{graphs_root}/metrics.json', 'w') as json_data:
        json.dump(metrics, json_data, indent=4)


1. /content/drive/MyDrive/BETA SKELETON LUNA Y CHUCHI/grafos/DC_m/10/DC_m_000.graphml -- Vertices 4,495 -- Aristas 33,582
Calcular 'fractal_dimension'
Tiempo: 2.0368835069998568 segundos

2. /content/drive/MyDrive/BETA SKELETON LUNA Y CHUCHI/grafos/DC_m/101/DC_m_000.graphml -- Vertices 4,611 -- Aristas 34,473
Calcular 'fractal_dimension'
Tiempo: 1.9551171419998354 segundos

3. /content/drive/MyDrive/BETA SKELETON LUNA Y CHUCHI/grafos/DC_m/0/DC_m_000.graphml -- Vertices 4,613 -- Aristas 34,364
Calcular 'fractal_dimension'
Tiempo: 1.691731437999806 segundos

4. /content/drive/MyDrive/BETA SKELETON LUNA Y CHUCHI/grafos/DC_m/100/DC_m_000.graphml -- Vertices 4,559 -- Aristas 33,875
Calcular 'fractal_dimension'
Tiempo: 1.467847232999702 segundos

5. /content/drive/MyDrive/BETA SKELETON LUNA Y CHUCHI/grafos/DC_m/1/DC_m_000.graphml -- Vertices 4,527 -- Aristas 33,599
Calcular 'fractal_dimension'
Tiempo: 1.1399880019998818 segundos

6. /content/drive/MyDrive/BETA SKELETON LUNA Y CHUCHI/grafos/D

## Prueba cálculo de métricas

Las métricas que nos interesan son las siguientes:
- De eficiencia: Global Efficiency, Local Efficiency y Average Local Efficiency.
- De centralidad: Closeness, Betweenness, Eigenvector y Convergence Degree.
- Espectrales: Obtener matriz laplaciana.
- Entropia.
- Fractalidad: Obtener la dimensión fractal (utilizando el método de box counting).


In [ ]:
# Archivo de ejemplo para calcular métricas
ex_graph_path = f'{graphs_root}/h_m/100/h_m_001.graphml'

t0 = time.perf_counter()
g = ig.Graph.Read_GraphML(ex_graph_path)  # Pruebas con igraph
print(f'Tiempo leyendo el grafo en igraph: {time.perf_counter() - t0}')

t0 = time.perf_counter()
G = ig.Graph.to_networkx(g)  # Para pruebas con networkx
print(f'Tiempo convirtiendo a networkx: {time.perf_counter() - t0}')

# No hay una implementación directa de la métrica de eficiencia en iGraph, pero sí en Networkx
# entonces me gustaría testear qué tal va en tiempo ambas.
# Hasta el momento, el tiempo de iGraph a Networkx no es tampoco tan grave, entonces puede ser viable

print(f'No. Nodos: {len(g.vs):,}')
print(f'No. Aristas: {len(g.es):,}')

Tiempo leyendo el grafo en igraph: 1.821109025999931
Tiempo convirtiendo a networkx: 2.0785899689999496
No. Nodos: 43,259
No. Aristas: 325,423


Para no perder el hilo de mis pensamientos, creo que será necesario utilizar varios nucleos del procesador y quizás el uso de memoria virtual con ayuda de dask

### Métricas de eficiencia

En igraph está [aquí](https://igraph.org/c/doc/igraph-Structural.html#efficiency-measures) las métricas que nos servirían en todo caso. La documentación está en C, pero puede sirve de igual forma

En networkx está aquí [aquí](https://networkx.org/documentation/stable/reference/algorithms/efficiency_measures.html). Afortunadamente, aquí sí se puede implementar directamente en python

#### Global Efficiency of the graph

En igraph no hay una implementación directa, así que tocó hacerla de cero. En cambio, en networkx sí la hay ([aquí](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.efficiency_measures.global_efficiency.html#networkx.algorithms.efficiency_measures.global_efficiency))

In [ ]:
def global_efficiency(g: ig.Graph) -> float:
    shortest_paths = g.distances()
    efficiency = 0.0
    n = len(g.vs)

    for i in range(n):
        for j in range(i+1, n):
            if shortest_paths[i][j] > 0:
                efficiency += 1/shortest_paths[i][j]

    efficiency /= (n * (n - 1) / 2)

    return efficiency


In [ ]:
#t0 = time.perf_counter()
#efficiency = global_efficiency(g)
#print(f'Tiempo total: {time.perf_counter() - t0}. Eficiencia: {efficiency}')

t0 = time.perf_counter()
efficiency = nx.global_efficiency(G)
print(f'Tiempo total: {time.perf_counter() - t0}. Eficiencia: {efficiency}')


KeyboardInterrupt: 

Networkx es una puta mierda bien lenta XD. Sin duda el ganador acá es igraph, además de que es excelente confirmar que las métricas sí esté dando igual en ambas. ME ENCANTA!

#### Local Efficiency Around Each Vertex

En networkx no encontré una así directa, existe una función parecida, pero no es exactamente la misma, en cualquier caso, es [esta](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.efficiency_measures.efficiency.html#networkx.algorithms.efficiency_measures.efficiency), pero no me convence y no veo tan clara su implementación siendo honesto

In [ ]:
def local_efficiency(g: ig.Graph, v: ig.Vertex):
    neighbors = g.neighbors(v)
    subgraph = g.subgraph(neighbors)

    if len(neighbors) <= 1:
        return 0

    return global_efficiency(subgraph)


In [ ]:
t0 = time.perf_counter()
local_efficiencies = [local_efficiency(g, v.index) for v in g.vs]
print(f"Tiempo: {time.perf_counter() - t0}. Eficiencias locales -- Máx {max(local_efficiencies)} -- Min {min(local_efficiencies)}")

Tiempo: 0.752547748999973. Eficiencias locales -- Máx 0.95 -- Min 0.49164750957854464


#### Average Local Efficiency

En igraph [aquí](https://igraph.org/c/doc/igraph-Structural.html#igraph_average_local_efficiency).  
En networkx [aquí](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.efficiency_measures.local_efficiency.html#networkx.algorithms.efficiency_measures.local_efficiency).

In [ ]:
#t0 = time.perf_counter()
#avg_local_efficiency = sum(local_efficiencies) / len(local_efficiencies)
#print(f"Tiempo: {time.perf_counter() - t0}. Eficiencias local promedio: {avg_local_efficiency}")

t0 = time.perf_counter()
local_efficiency = nx.local_efficiency(G)
print(f"Tiempo: {time.perf_counter() - t0}. Eficiencia local: {local_efficiency}")


EXCELENTE!!!!!!! igraph es la absoluta verga en cuanto a velocidad se refiere. Y lo que me pone más contento es que ya confirmamos que a pesar de que estemos creando nuestras propias funciones para calcular nuestras métricas, todo esté saliendo excelente! Estoy muy contento sinceramente, espero que puedas sentir lo mismo que yo viendo este colab

### Metricas de centralidad

Aquí lo bueno que es que afortunadamente, en igraph sí se pueden calcular perfectamente estas métricas sin problemas, al igual que en networkx

#### Closeness

igraph. [Aquí](https://python.igraph.org/en/stable/api/igraph.GraphBase.html#closeness).  
networkx. [Aquí](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.closeness_centrality.html#networkx.algorithms.centrality.closeness_centrality).

In [ ]:
t0 = time.perf_counter()
closeness = g.closeness(weights='distance', normalized=True)
print(f'Tiempo {time.perf_counter() - t0}. Closeness -- min {min(closeness)} -- máx {max(closeness)}')

t0 = time.perf_counter()
closeness = nx.closeness_centrality(G, distance='distance', wf_improved=False)
print(f'Tiempo {time.perf_counter() - t0}. Closeness -- min {min(closeness.values())} -- máx {max(closeness.values())}')

XD

#### Betweenness

igraph. [Aquí](https://python.igraph.org/en/stable/api/igraph.GraphBase.html#betweenness).  
networkx. [Aquí](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.betweenness_centrality.html#networkx.algorithms.centrality.betweenness_centrality)

In [ ]:
# Así el pc no me da pa hacerlo XD. Raramente se me apaga cuando está cuando está con networkx xd
# sin embargo, sí dan lo mismo, ya lo confirmé! Antes lo ejecuté y se demoraba ~12 mins

t0 = time.perf_counter()
betweenness = g.betweenness(directed=False, weights='distance')
print(f'Tiempo {time.perf_counter() - t0}. Betweenness -- min {min(betweenness)} -- máx {max(betweenness)}')

t0 = time.perf_counter()
betweenness = nx.betweenness_centrality(G, weight='distance', normalized=False)
print(f'Tiempo {time.perf_counter() - t0}. Betweenness -- min {min(betweenness.values())} -- máx {max(betweenness.values())}')

Tiempo 11.998107334000451. Betweenness -- min 0.0 -- máx 157568.0


KeyboardInterrupt: 

#### Eigenvector

igraph. [Aquí](https://python.igraph.org/en/stable/api/igraph.GraphBase.html#eigenvector_centrality).  
networkx. [Aquí](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.eigenvector_centrality.html#networkx.algorithms.centrality.eigenvector_centrality).

In [ ]:
t0 = time.perf_counter()
eigenvector = g.eigenvector_centrality(directed=False, weights='distance', scale=False)
print(f'Tiempo {time.perf_counter() - t0}. Eigenvector -- min {min(eigenvector)} -- máx {max(eigenvector)}')

t0 = time.perf_counter()
eigenvector = nx.eigenvector_centrality(G, weight='distance')
print(f'Tiempo {time.perf_counter() - t0}. Eigenvector -- min {min(eigenvector.values())} -- máx {max(eigenvector.values())}')

Tiempo 0.043060715001047356. Eigenvector -- min 2.0751151911796389e-10 -- máx 0.36281617016534623
Tiempo 3.7051847289985744. Eigenvector -- min 2.1215184078790071e-10 -- máx 0.362605268407333


Qué chévere, bastante rápida ambas. Pero sigue siendo mejor mi igraph querido

#### Convergence

En igraph la función sí se encuentra (ver [aquí](https://python.igraph.org/en/stable/api/igraph.GraphBase.html#convergence_degree)), pero está el grandisimo problema de que no está documentada, por tanto, me tocará conformarme con los parametros que salen mencionados en la implementación en C ([aquí](https://igraph.org/c/doc/igraph-Structural.html#igraph_convergence_degree)). Parametros los cuales tampoco aplicarían mucho en este caso realmente, entonces supongo que se irá sin parametros XD

En networkx es peor, porque la gran puta función no está

In [ ]:
t0 = time.perf_counter()
convergence = g.convergence_degree()
print(f'Tiempo {time.perf_counter() - t0}. convergence -- min {min(convergence)} -- máx {max(convergence)}')

Tiempo 5.98602589300026. convergence -- min 0.0 -- máx 0.9995595683770094


### Espectrales

igraph. [Aquí](https://python.igraph.org/en/stable/api/igraph.GraphBase.html#laplacian).  
networkx. [Aquí](https://networkx.org/documentation/stable/reference/generated/networkx.linalg.laplacianmatrix.laplacian_matrix.html#networkx.linalg.laplacianmatrix.laplacian_matrix).

Para tener en cuenta, la matriz laplaciana de networkx está utilizando un objeto de la clase scipy, en concreto, este de [acá](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_array.html).

In [ ]:
t0 = time.perf_counter()
laplacian_g = g.laplacian(weights='distance', mode='all')
tf = time.perf_counter() - t0
print(f'Tiempo: {tf}. min: {min(map(min, laplacian_g))} x {max(map(max, laplacian_g))}')

t0 = time.perf_counter()
laplacian_G = nx.laplacian_matrix(G, weight='distance')
print(f'Tiempo: {time.perf_counter() - t0}. min: {laplacian_G.min()} -- max: {laplacian_G.max()}')

Tiempo: 25.71713759799968. min: -1101.1961354574396 -- max: 11391.87880081494


Ve, por primera vez ganó networkx xd. Excelente saberlo en realidad, además de confirmar que den lo mismo.

En este caso, va a ser completamente necesario utilizar networkx para calcular esta métrica en particular, puesto que se crashea la sesión si se utiliza igraph por el número de nodos, ya que, esta matriz laplaciana queda con la forma de $N \times N$, y si utilizamos $400.000$ nodos (lo cual no es exagerado), pasa lo que pasa. Entonces, la conversión a networkx será necesaria aunque cueste su debido tiempo.

### Entropia & Fractalidad

En estas métrica de acá no hay documentación por parte de ambas librerias, pero sí de scipy en el caso de la entropia ([aquí](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.entropy.html)), pero no me gustaría confirmar con David cómo quiere implementarla exactamente, porque no lo tengo tan claro en realidad. En el código que me envió, calcula la entropia en la distribución de los grados de cada nodo.

Y, en el caso de la fractalidad, no lo tengo tampoco tan claro, entonces sería bueno verlo también. Por mientras, trabajaré así sin estas dos métricas hasta que ya se tenga algo más claro. Cualquier cosa lo colocaré acá!!